# SingleStoreDB vector search
SingleStore DB是一个高性能的分布式数据库，支持云端和本地部署。其长期以来一直提供矢量函数（如[内积](https://docs.singlestore.com/managed-service/en/reference/sql-reference/vector-functions/dot_product.html)）的支持，因此成为需要文本相似度匹配的人工智能应用的理想解决方案。
本教程介绍了如何利用SingleStore DB Vector Store的功能。

In [ ]:
# Establishing a connection to the database is facilitated through the singlestoredb Python connector.
# Please ensure that this connector is installed in your working environment.
!pip install singlestoredb

In [ ]:
import os
import getpass

# We want to use OpenAIEmbeddings so we have to get the OpenAI API Key.
os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')


In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import SingleStoreDB
from langchain.document_loaders import TextLoader

In [ ]:
# Load text samples 
from langchain.document_loaders import TextLoader
loader = TextLoader('../../../state_of_the_union.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

建立[连接](https://singlestoredb-python.labs.singlestore.com/generated/singlestoredb.connect.html)到数据库可以有几种方式。您可以设置环境变量或将命名参数传递给`SingleStoreDB构造函数`。此外，您还可以将这些参数提供给`from_documents`和`from_texts`方法。

In [ ]:
# Setup connection url as environment variable
os.environ['SINGLESTOREDB_URL'] = 'root:pass@localhost:3306/db'

# Load documents to the store
docsearch = SingleStoreDB.from_documents(
    docs,
    embeddings,
    table_name = "noteook", # use table with a custom name 
)

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
docs = docsearch.similarity_search(query) # Find documents that correspond to the query
print(docs[0].page_content)